<a href="https://colab.research.google.com/github/merve-erdem01/merve-erdem/blob/main/hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [73]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity


In [87]:
def load_data():
  joke_text = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/JokeText.csv')

  user_ratings1= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UserRatings1.csv')
  user_ratings2= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UserRatings2.csv')

  user_ratings = pd.concat([user_ratings1, user_ratings2], axis=1)
  user_ratings.fillna(0, inplace=True)
  return joke_text, user_ratings

In [83]:
def preprocess_data(user_ratings):
    ratings_matrix = user_ratings.drop(columns=['JokeId']).values
    return ratings_matrix

In [84]:
def train_svd_model(ratings_matrix, k=20):
    U, sigma, Vt = svds(ratings_matrix, k=k)
    sigma = np.diag(sigma)
    predicted_ratings = np.dot(np.dot(U, sigma), Vt)
    return predicted_ratings

# Recommend Jokes
def recommend_jokes(user_id, predicted_ratings, joke_text, num_recommendations=5):
    user_index = user_id - 1  # User IDs are 1-based in the dataset
    user_ratings_pred = predicted_ratings[user_index]
    joke_indices = np.argsort(user_ratings_pred)[::-1][:num_recommendations]
    recommended_jokes = [joke_text.iloc[joke_id]['JokeText'] for joke_id in joke_indices if joke_id < len(joke_text)]
    return recommended_jokes

In [85]:
def find_jokes_by_keyword(keyword, joke_text):
    return joke_text[joke_text['JokeText'].str.contains(keyword, case=False, na=False)]

# Recommend jokes based on keyword
def recommend_jokes_by_keyword(keyword, joke_text, user_id, predicted_ratings, num_recommendations=5):
    jokes_with_keyword = find_jokes_by_keyword(keyword, joke_text)
    if jokes_with_keyword.empty:
        return []
    joke_ids = jokes_with_keyword.index.tolist()
    user_index = user_id - 1
    user_ratings_pred = predicted_ratings[user_index]
    joke_indices = [idx for idx in np.argsort(user_ratings_pred)[::-1] if idx in joke_ids]
    recommended_jokes = [joke_text.iloc[joke_id]['JokeText'] for joke_id in joke_indices[:num_recommendations]]
    return recommended_jokes


In [89]:
# Main Function
if __name__ == "__main__":
    joke_text, user_ratings = load_data()
    ratings_matrix = preprocess_data(user_ratings)
    predicted_ratings = train_svd_model(ratings_matrix, k=20)

    # Input keyword
    keyword = input("Enter a keyword to search for jokes: ")
    recommended_jokes_user_1 = recommend_jokes_by_keyword(keyword, joke_text, 1, predicted_ratings, 5)

    if recommended_jokes_user_1:
        print(f"Recommended Jokes containing the keyword '{keyword}' for User 1:")
        for joke in recommended_jokes_user_1:
            print(joke)
    else:
        print(f"No jokes containing the keyword '{keyword}' were found.")

Enter a keyword to search for jokes: doctor
Recommended Jokes containing the keyword 'doctor' for User 1:
A man visits the doctor. The doctor says "I have bad news for you.You have
cancer and Alzheimer's disease". 
The man replies "Well,thank God I don't have cancer!"

A Czechoslovakian man felt his eyesight was growing steadily worse, and 
felt it was time to go see an optometrist. 

The doctor started with some simple testing, and showed him a standard eye 
chart with letters of
diminishing size: CRKBNWXSKZY. . . 

"Can you read this?" the doctor asked. 

"Read it?" the Czech answered. "Doc, I know him!"

A man, recently completing a routine physical examination receives a
phone call from his doctor.  The doctor says, "I have some good news and
some bad news."  The man says, "OK, give me the good news first."  The
doctor says, "The good news is, you have 24 hours to live."  The man
replies, "Shit!  That's the good news?  Then what's the bad news?"

The doctor says, "The bad news is, 

In [20]:
print("JokeText columns:", joke_text.columns)
print("UserRatings columns:", user_ratings.columns)

JokeText columns: Index(['JokeId', 'JokeText'], dtype='object')
UserRatings columns: Index(['JokeId', 'User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7',
       'User8', 'User9',
       ...
       'User73412', 'User73413', 'User73414', 'User73415', 'User73416',
       'User73417', 'User73418', 'User73419', 'User73420', 'User73421'],
      dtype='object', length=73423)


In [21]:
joke_text.head()


,JokeId,JokeText
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...


In [10]:
joke_text.shape

(100, 2)

In [11]:
joke_text.drop_duplicates(subset ='JokeText', keep = 'first', inplace = True)
joke_text.shape

(100, 2)

This shows there we no duplicates in our data

I wanted to improve the code by calling connotations words as well. Previous part is only giving results for the words that are in the joke but I want it to work like recommend jokes related to similar words such as if I say "baloon" it can also search for "fly", "burst" etc

In [90]:
pip install gensim

In [91]:


from gensim.models import KeyedVectors
from gensim.downloader import load



In [93]:
def load_word2vec_model():
    model = load("word2vec-google-news-300")
    return model

In [94]:
def find_jokes_by_keyword_and_related_words(keyword, joke_text, word2vec_model):
    # Find related words using word2vec
    related_words = [keyword]
    if keyword in word2vec_model:
        related_words += [w for w, _ in word2vec_model.most_similar(keyword, topn=10)]
    # Search for jokes containing any of the related words
    joke_mask = joke_text['JokeText'].apply(lambda x: any(word in x for word in related_words))
    return joke_text[joke_mask]

# Recommend jokes based on keyword and related words
def recommend_jokes_by_keyword(keyword, joke_text, user_id, predicted_ratings, word2vec_model, num_recommendations=5):
    jokes_with_keyword = find_jokes_by_keyword_and_related_words(keyword, joke_text, word2vec_model)
    if jokes_with_keyword.empty:
        return []
    joke_ids = jokes_with_keyword.index.tolist()
    user_index = user_id - 1
    user_ratings_pred = predicted_ratings[user_index]
    joke_indices = [idx for idx in np.argsort(user_ratings_pred)[::-1] if idx in joke_ids]
    recommended_jokes = [joke_text.iloc[joke_id]['JokeText'] for joke_id in joke_indices[:num_recommendations]]
    return recommended_jokes

# Main Function
if __name__ == "__main__":
    word2vec_model = load_word2vec_model()
    joke_text, user_ratings = load_data()
    ratings_matrix = preprocess_data(user_ratings)
    predicted_ratings = train_svd_model(ratings_matrix, k=20)

    # Input keyword
    keyword = input("Enter a keyword to search for jokes: ")
    recommended_jokes_user_1 = recommend_jokes_by_keyword(keyword, joke_text, 1, predicted_ratings, word2vec_model, 5)

    if recommended_jokes_user_1:
        print(f"Recommended Jokes containing the keyword '{keyword}' or related words for User 1:")
        for joke in recommended_jokes_user_1:
            print(joke)
    else:
        print(f"No jokes containing the keyword '{keyword}' or related words were found.")

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Enter a keyword to search for jokes: baloon
Recommended Jokes containing the keyword 'baloon' or related words for User 1:
A man piloting a hot air balloon discovers he has wandered off course and
is hopelessly lost. He descends to a lower altitude and locates a man
down on the ground. He lowers the balloon further and shouts "Excuse me,
can you tell me where I am?"

The man below says: "Yes, you're in a hot air balloon, about 30 feet
above this field."

"You must work in Information Technology," says the balloonist.

"Yes I do," replies the man. "And how did you know that?"

"Well," says the balloonist, "what you told me is technically correct,
but of no use to anyone."

The man below says, "You must work in management."

"I do," replies the balloonist, "how did you know?"

"Well," says the man, "you don't know where you are, or where you're
going, but you expect my immediate help. You're in the sam